# 1. 라이브러리 및 API key 설정

In [1]:
import ast
import pandas as pd
df = pd.read_csv("/Users/yoonjincho/Desktop/윤진 2025/YAI/YAI_personal_archive/NLP_ToyProject/DataGeneration/dataset_0302_with_selected_sentence_1386개.csv")
df

,Unnamed: 0,news,extracted_entities,wiki_summary,wiki_full,label,title,news_merged,key sentences,key sentences explanation,summarized evidence sentences
0,0,u.s. senate republican leader mitch mcconnell ...,"Mitch McConnell, Myanmar, Aung San Suu Kyi, Jo...",Mitch McConnell: Addison Mitchell McConnell II...,Mitch McConnell: Addison Mitchell McConnell II...,1,senate leader opposes lecturing myanmar leader...,title : senate leader opposes lecturing myanma...,"[""Mitch McConnell said on Tuesday he would not...","[""McConnell's opposition to the resolution dir...","[""Mitch McConnell is an American politician an..."
1,2,note little johnny might want consider use dia...,"little johnny, isis, fort riley military base,...",little johnny: Little Johnny jokes are about a...,little johnny: Little Johnny jokes are about a...,1,NaN,content : note little johnny might want consid...,"[""A 20-year-old Kansas man plotted to kill an ...","[""This sentence supports the news because it p...","[""Little Johnny jokes are about a small boy wh..."
2,3,president obama is clearly in good spirits tha...,"president obama, republican party, trump presi...",president obama: Barack Hussein Obama II (born...,president obama: Barack Hussein Obama II (born...,0,president obama gets his troll game on by than...,title : president obama gets his troll game on...,"[""Obama said it was crucial for voters to oppo...","[""This sentence supports the news because it e...","[""President Obama is the 44th president of the..."
3,5,a racist white woman in florida is surely furi...,"Florida, Tallahassee, Trump, Martin Luther Kin...",Florida: Florida ( FLORR-ih-də; Spanish: [flo...,Florida: Florida ( FLORR-ih-də; Spanish: [flo...,0,racist florida woman tells black people you sh...,title : racist florida woman tells black peopl...,[],[],"[""Florida is a state in the Southeastern regio..."
4,6,anyone question whether bundy-led right-wing t...,"Bundy, Malheur National Wildlife Refuge, Amand...",Bundy: Theodore Robert Bundy (né Cowell; Novem...,Bundy: Theodore Robert Bundy (né Cowell; Novem...,1,NaN,content : anyone question whether bundy-led ri...,"[""On January 2, 2016, an armed group of right-...","[""This sentence supports the news because it f...","[""On January 2, 2016, an armed group of right-..."
...,...,...,...,...,...,...,...,...,...,...,...
1381,268,washington u.s. senate expect vote 11 a.m. edt...,"Donald Trump, Neil Gorsuch, U.S. Senate, U.S. ...","Donald Trump: Donald John Trump (born June 14,...","Donald Trump: Donald John Trump (born June 14,...",0,NaN,content : washington u.s. senate expect vote 1...,"[""Donald Trump nominated Neil Gorsuch to the S...","[""This sentence supports the news because it d...","[""Donald John Trump is an American politician ..."
1382,269,this is huge! the doj has authorized the fbi i...,"FBI, DOJ, Hillary Clinton, Loretta Lynch, Vict...",FBI: The Federal Bureau of Investigation (FBI)...,FBI: The Federal Bureau of Investigation (FBI)...,0,breaking news on clinton informants gag order ...,title : breaking news on clinton informants ga...,"[""The DOJ has authorized the FBI informant to ...","[""The first sentence supports the news because...","[""The Federal Bureau of Investigation (FBI) is..."
1383,270,washington u.s. house speaker paul ryan thursd...,"Paul Ryan, Washington, U.S. House, National Fo...","Paul Ryan: Paul Davis Ryan (born January 29, 1...","Paul Ryan: Paul Davis Ryan (born January 29, 1...",0,NaN,content : washington u.s. house speaker paul r...,"[""U.S. House Speaker Paul Ryan said NFL player...","[""The first sentence supports the news because...","[""Paul Ryan is an American politician who serv..."
1384,271,dinesh d sousa warned us about obama s reducti...,"Dinesh D'Souza, Barack Obama, China, Russia, N...",Dinesh D'Souza: Dinesh Joseph D'Souza (; born ...,Dinesh D'Souza: Dinesh Joseph D'Souza (; born ...,0,obama undermines americaplans to slash nuclear...,title : obama undermines americaplans to slash...,"[""The Unite

In [5]:
import pandas as pd
import ast
import json
# 이미 df가 주어졌다고 가정
# 예: df = pd.read_csv("input.csv")
df = df.iloc[0:500, :]
df['gold'] = ''  # 'c' 열이 없거나 초기화가 필요한 경우

for i, row in df.iterrows():
    try:
        a_list = ast.literal_eval(row['key sentences'])
        b_list = ast.literal_eval(row['summarized evidence sentences'])
        print(a_list)
        indices=extract_sentences(a_list,b_list)
        print(indices)         
        df.at[i, 'gold'] = str(indices['index'])
    except ValueError:
        # a_list 중 하나라도 b_list 내에 없으면 빈 문자열 처리
        df.at[i, 'gold'] = ''

# 수정된 DataFrame을 다시 저장
df.to_csv('output_0-500.csv', index=False)


/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_69077/2009984077.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gold'] = ''  # 'c' 열이 없거나 초기화가 필요한 경우


["Mitch McConnell said on Tuesday he would not support a resolution targeting Myanmar leader Aung San Suu Kyi over the treatment of the country's Rohingya Muslims.", "I don't favor a resolution going after her, McConnell, who has been engaged with issues related to Myanmar, also known as Burma, for years.", "My personal view is America kind of singling her out, and lecturing her when she's in a very challenging position is not helpful.", 'Senators John McCain, a Republican, and Richard Durbin, a Democrat, introduced a resolution last week condemning the violence and urging Suu Kyi to act.', 'International pressure has been mounting on Myanmar to end violence that has sent about 370,000 Rohingya Muslims fleeing to Bangladesh.', "While Washington has been a staunch supporter of Myanmar's transition from decades of harsh military rule being led by Suu Kyi, the Nobel laureate has been criticized as doing too little to stop the violence."]
{'index': [2, 3, 8, 6, 9, 14]}
['A 20-year-old Kans

In [2]:
import os
import torch
import numpy as np
import datasets
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import roc_auc_score
from openai import OpenAI
from serpapi import GoogleSearch 
from dotenv import load_dotenv
import pandas as pd
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langchain_anthropic import ChatAnthropic, AnthropicLLM
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.memory import SimpleMemory
from datetime import datetime
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import json
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()


/opt/anaconda3/envs/nlp_1/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# OpenAI 및 SerpAPI 키 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERP_API_KEY = os.getenv("SERPAPI_API_KEY")
# OpenAI 클라이언트 초기화
client = OpenAI(api_key=OPENAI_API_KEY)

# FAISS 인덱스 저장 경로
FAISS_INDEX_PATH = Path('local/news-please/faiss_index')

/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_69077/2488759682.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=OPENAI_API_KEY)


# 2. 요약


In [4]:
import os
import json
import wikipedia
import nltk
import pandas as pd

# LangChain 관련
from langchain.llms import OpenAI  # 예시로 OpenAI 사용
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 의학과학개요 검토 LLM
llm1 = ChatOpenAI(temperature=1,               # 창의성 (0.0 ~ 2.0) 
                 max_tokens=2048,             # 최대 토큰수
                 model_name='gpt-4o-mini',  # 모델명
                )

llm2 = ChatAnthropic(model='claude-3-5-sonnet-20240620',max_tokens=8192)

########################################
# 1. LangChain을 이용한 키워드 추출 함수
########################################
import json

def extract_sentences(a, b) -> dict:
    """
    Extracts indices of key sentences (a) from a larger list of sentences (b).
    Ensures the return value is strictly JSON, with an "index" field.
    """

    prompt = PromptTemplate(
        input_variables=["a", "b"],
        template=(
            """
            Key sentences: {a}
            Full sentences: {b}

            Both 'key' and 'full' are lists containing multiple sentences. The key sentences may appear in the full list in slightly altered forms (i.e., they might not match exactly, but rather be paraphrased or partially different). 

            For example, if a key sentence is "Amy ate an apple," and the full list is ["Amy is a child.", "She ate an apple"], the returned indices should be [1].

            Please carefully compare the key sentences with each sentence in the full list. Assign indices to each sentence in the full list (0, 1, 2, ...), then figure out which full-list indices correspond to the key sentences.

            For each key sentence, choose only the single sentence from the full list that best matches its content. If two key sentences match with same sentence, drop duplicates in the output.

            Your output must be in valid JSON format only, with no additional text or explanations. Use the following structure:
            <output struncture>
            {{"index": [0, 5, 15, 30]}}
            </output struncture>
            """
        )
    )

    chain = prompt | llm2 | StrOutputParser()
    try:
        response_text = chain.invoke({"a": a, "b": b})
        
        # Ensure the response starts and ends with braces
        if not response_text.startswith("{") or not response_text.endswith("}"):
            raise ValueError("Invalid JSON response from LLM")

        response_json = json.loads(response_text)
        return response_json

    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        print("Raw LLM Response:", response_text)
        return {"error": "Invalid JSON format returned from LLM"}

In [52]:
df = data.iloc[0:100,:]
df

,news,extracted_entities,wiki_summary,wiki_full,label,title,news_merged
0,u.s. senate republican leader mitch mcconnell ...,"Mitch McConnell, Myanmar, Aung San Suu Kyi, Jo...",Mitch McConnell: Addison Mitchell McConnell II...,Mitch McConnell: Addison Mitchell McConnell II...,1,senate leader opposes lecturing myanmar leader...,title : senate leader opposes lecturing myanma...
1,four u.s. senators - two democrats and two rep...,"senators charles grassley, senator patrick lea...",senators charles grassley: Charles Ernest Gras...,senators charles grassley: Charles Ernest Gras...,1,senators introduce bill aimed at getting gener...,title : senators introduce bill aimed at getti...
2,note little johnny might want consider use dia...,"little johnny, isis, fort riley military base,...",little johnny: Little Johnny jokes are about a...,little johnny: Little Johnny jokes are about a...,1,NaN,content : note little johnny might want consid...
3,president obama is clearly in good spirits tha...,"president obama, republican party, trump presi...",president obama: Barack Hussein Obama II (born...,president obama: Barack Hussein Obama II (born...,0,president obama gets his troll game on by than...,title : president obama gets his troll game on...
4,turkey said on monday its former economy minis...,"Zafer Caglayan, Bekir Bozdag, FETO, Tayyip Erd...",Zafer Caglayan: Mehmet Zafer Çağlayan (born 10...,Zafer Caglayan: Mehmet Zafer Çağlayan (born 10...,1,turkey says u.s. indictment of former minister...,title : turkey says u.s. indictment of former ...
...,...,...,...,...,...,...,...
95,a suicide car bomber blew himself up at a secu...,"Islamic State, Houthi rebels, Yemen, Al Qaeda ...","Islamic State: The Islamic State (IS), also kn...","Islamic State: The Islamic State (IS), also kn...",1,car bomber kills at least 15 in yemeni port ad...,title : car bomber kills at least 15 in yemeni...
96,london british prime minister theresa may rais...,"Theresa May, Benjamin Netanyahu, London, Israe...","Theresa May: Theresa Mary May, Baroness May of...","Theresa May: Theresa Mary May, Baroness May of...",0,NaN,content : london british prime minister theres...
97,the obama administration s cleansing of the is...,"Omar Mateen, Abu Bakr al-Baghdadi, Paul D. Rya...",Omar Mateen: Omar Mir Seddique Mateen (Pashto:...,Omar Mateen: Omar Mir Seddique Mateen (Pashto:...,0,two words obamas doj has been hiding from orla...,title : two words obamas doj has been hiding f...
98,the intimidation tactics of mueller s fbi are ...,"Robert Mueller, Paul Manafort, Andrew Weissman...",Robert Mueller: Robert Swan Mueller III (; bor...,Robert Mueller: Robert Swan Mueller III (; bor...,0,muellers fbi raid of paul manafort and wife pr...,title : muellers fbi raid of paul manafort and...


In [57]:
import pandas as pd
import json

# 데이터 로드 및 초기화
data = pd.read_csv("/Users/yoonjincho/Desktop/윤진 2025/YAI/YAI_personal_archive/NLP_ToyProject/merged_data_0302_2차.csv").drop(columns=['Unnamed: 0'])
output_path = "output_0302_0-100.csv"  # 결과 CSV 파일명
df = data.iloc[0:100, :]

# 결과 컬럼 추가 (초기화)
df["key sentences"] = None
df["key sentences explanation"] = None
df["summarized evidence sentences"] = None

# 각 행 처리
for idx, row in df.iterrows():
    news_text = row["news_merged"]
    news_veracity = row["label"]  # 라벨 컬럼명 확인 필요
    evidence = row["wiki_summary"]

    try:
        # 뉴스에서 키워드(엔티티) 추출
        response = extract_sentences(news_text, news_veracity, evidence)

        # 응답이 유효한 JSON 객체인지 확인
        if isinstance(response, dict) and "key sentences" in response and "key sentences explanation" in response and "summarized evidence sentences" in response:
            df.at[idx, "key sentences"] = json.dumps(response["key sentences"], ensure_ascii=False)
            df.at[idx, "key sentences explanation"] = json.dumps(response["key sentences explanation"], ensure_ascii=False)
            df.at[idx, "summarized evidence sentences"] = json.dumps(response["summarized evidence sentences"], ensure_ascii=False)
        else:
            print(f"⚠️ Warning: Unexpected response format at index {idx}")
    
    except Exception as e:
        print(f"❌ Error at index {idx}: {e}")

    # 10의 배수일 때마다 파일 저장
    if (idx + 1) % 10 == 0:
        df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"✅ Progress Saved: {output_path} at index {idx}")

# 최종 저장
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"🎉 Processing complete. Final output saved to '{output_path}'")

/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_88251/2834649438.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["key sentences"] = None
/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_88251/2834649438.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["key sentences explanation"] = None
/var/folders/4y/pfdhlm_n2h5bggss4w_pv7cr0000gn/T/ipykernel_88251/2834649438.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

❌ Error at index 1: Invalid JSON response from LLM
❌ Error at index 4: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 9
❌ Error at index 18: Invalid JSON response from LLM
❌ Error at index 19: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 19
✅ Progress Saved: output_0302_0-100.csv at index 29
✅ Progress Saved: output_0302_0-100.csv at index 39
❌ Error at index 47: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 49
✅ Progress Saved: output_0302_0-100.csv at index 59
✅ Progress Saved: output_0302_0-100.csv at index 69
✅ Progress Saved: output_0302_0-100.csv at index 79
❌ Error at index 83: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 89
❌ Error at index 99: Invalid JSON response from LLM
✅ Progress Saved: output_0302_0-100.csv at index 99
🎉 Processing complete. Final output saved to 'output_0302_0-100.csv'


In [56]:
df

,news,extracted_entities,wiki_summary,wiki_full,label,title,news_merged,key sentences,key sentences explanation,summarized evidence sentences
0,u.s. senate republican leader mitch mcconnell ...,"Mitch McConnell, Myanmar, Aung San Suu Kyi, Jo...",Mitch McConnell: Addison Mitchell McConnell II...,Mitch McConnell: Addison Mitchell McConnell II...,1,senate leader opposes lecturing myanmar leader...,title : senate leader opposes lecturing myanma...,"[""U.S. Senate Republican leader Mitch McConnel...","[""This sentence supports the news because it d...","[""Mitch McConnell is the senior United States ..."


In [25]:
summary

'Myanmar, officially the Republic of the Union of Myanmar and also rendered as Burma (the official English form until 1989), is a country in northwest Southeast Asia. It is the largest country by area in Mainland Southeast Asia and has a population of about 55 million. It is bordered by India and Bangladesh to its northwest, China to its northeast, Laos and Thailand to its east and southeast, and the Andaman Sea and the Bay of Bengal to its south and southwest. The country\'s capital city is Naypyidaw, and its largest city is Yangon (formerly Rangoon).\nEarly civilisations in the area included the Tibeto-Burman-speaking Pyu city-states in Upper Myanmar and the Mon kingdoms in Lower Myanmar. In the 9th century, the Bamar people entered the upper Irrawaddy valley, and following the establishment of the Pagan Kingdom in the 1050s, the Burmese language and culture and Theravada Buddhism slowly became dominant in the country. The Pagan Kingdom fell to Mongol invasions, and several warring s